# 总体架构

&emsp;&emsp;&emsp;&emsp;早期的机器阅读理解模型大多基于检索技术，但是信息检索主要依赖关键词的匹配，而在很多情况下，单纯依靠问题和文章片段的文字匹配找到的答案与问题并不相关。随着深度学习的发展，机器阅读理解步入了神经网络时代。本文将介绍基于深度学习的机器阅读理解模型的架构，探究其提升性能的原因。

&emsp;&emsp;&emsp;&emsp;基于深度学习的MRC模型构造各异，但是经过多年的实践和探索，逐渐形成了稳定框架结构。本节将介绍MRC模型共有的总体架构。

&emsp;&emsp;&emsp;&emsp;MRC模型的输入为文章和问题。因此，首先要对这两部分进行数字化编码，将其变成可以被计算机处理的信息单元。在编码的过程中，模型需要保留原有语句在文章中的语义。因此，每个单词、短语和句子的编码必须建立在理解上下文的基础上。我们把模型中进行编码的模块称为编码层。

&emsp;&emsp;&emsp;&emsp;接下来，由于文章和问题之间存在相关性，所以模型需要建立文章和问题之间的联系。例如，如果问题中出现关键词“河流”，而文章中出现关键词“长江”。虽然两个词不完全一样，但是其语义编码接近。因此，文章中“长江”一词以及邻近的语句将成为模型回答问题时的重点关注对象。这可以通过注意力机制加以解决。在这个过程中，MRC模型将文章和问题的语义结合在一起进行考量，进一步加深模型对于两者的理解。我们将这个模块称为交互层。交互层可以让模型聚焦文章和问题的语义联系，借助于文章的语义分析加深对问题的理解，同时借助于问题的语义分析加深对文章的理解。

&emsp;&emsp;&emsp;&emsp;经过交互层，模型建立起文章和问题之间的语义联系，接下来就可以预测问题的答案了。完成预测功能的模块称为输出层。MRC任务的答案有多种类型，因此输出层的具体形式需要和任务的答案类型相关联。此外，输出层确定模型优化时的评估函数和损失函数。

# 编码层

&emsp;&emsp;&emsp;&emsp;与其他基于深度学习的自然语言处理模型类似，MRC模型首先需要将文字形式的文章和问题转化成词向量。编码层一般采用类似的分布式算法对文本进行分词和向量化处理，然后加入字符编码等更丰富的信息，并采用上下文编码获得每个单词在具体语境中的含义。

## 词表的建立与初始化

&emsp;&emsp;&emsp;&emsp;首先，模型对训练文本进行分词以得到其中所有的单词。然后，根据阈值选取出现次数超过一定次数的单词组成词表，词表以外的单词视为非词表词（Out-Of-Vocabulary, OOV），用特殊单词<UNK\>表示。这样，模型可以得到一个大小为|V|的词表，每个词表中的词用一个d维向量表示。接下来，我们有两种方法获得词表中的单词向量：
    <br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;1、保持词表向量不变，即采用预训练词表中的向量（如Word2Vec的300维向量），在训练过程中不进行改变；
    <br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;2、将词表中的向量视为参数，在训练过程中和其他参数一起求导并优化。这里，既可以使用预训练词表向量进行初始化，也可以选择随机初始化。

&emsp;&emsp;&emsp;&emsp;第一种选择的优势是模型参数少，训练初期收敛较快；第二种选择的优势是可以根据实际数据调整词向量的值，以达到更好的训练效果。而采用预训练词表向量初始化一般可以使得模型在优化的最初几轮获得明显比随机初始化方法更优的结果。

&emsp;&emsp;&emsp;&emsp;在编码层中，为了更准确地表示每个单词在语句中的语义，除了词向量外，还经常对命名实体（named entity）和词性（part-of-speech）进行向量化处理。如果命名实体共有N种，则建立大小为N的命名实体表，每种命名实体用一个长度为d_N的向量表示；如果词性共有P种，则建立大小为P的词性表，每种词性用一个长度为d_P的向量表示。两个表中的向量均为可训练的参数。然后，用文本分析包，如spaCy，获得文章和问题中的每个词的命名实体和词性，再将对应向量拼接在词向量之后。由于一个词的命名实体属性与词性和这个词所在的语句有关，因此用这种方式获得向量编码可以更好地表示单词的语义，在许多模型中性能都有明显的提升。

&emsp;&emsp;&emsp;&emsp;另一种在机器阅读理解中非常有效的单词编码是精确匹配（exact matching）编码。exact matching编码适用于文章中的单词。对于文章中的单词w，检查w是否出现在问题中：如果是，w的精确匹配编码为1，否则为0，然后将这个二进制位拼接在单词向量后。精确匹配编码可以使模型快速找到文章中出现了问题单词的部分，而许多问题的答案往往就在这部分内容的附近。

## 字符编码

&emsp;&emsp;&emsp;&emsp;文本处理中时常会出现拼写错误，这时通过字符组合，往往可以识别正确的单词形式。此外，许多语言中存在词根的概念，即词的一部分是一个常见且有固定含义的子词。在单词的理解中，字符和子词具有很强的辅助作用。

&emsp;&emsp;&emsp;&emsp;为了更好地利用字符信息，在编码层中采用字符编码，即每个字符用一个向量表示。但是，由于单词的长度不一，每个单词可能有不同个数的字符向量。为了产生一个固定长度的字符信息向量，我们可以将多个字符向量合并成一个编码。最常用的模型是字符CNN。

&emsp;&emsp;&emsp;&emsp;设一个单词有K个字符，且对应的K个字符向量为（c_1, c_2, ..., c_k），每个向量维度为c。字符CNN利用一个窗口大小为W且有f个输出通道的CNN获得（K-W+1）个f维向量。然后，采用最大池化方法求得这些向量每一个维度上的最大值，形成一个f维向量作为结果。

In [2]:
# 字符卷积神经网络Char-CNN的PyTorch实现

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [40]:
class CharCNNMaxpooling(nn.Module):
    # char_num为字符表大小，char_dim为字符向量长度，window_size为CNN窗口长度，out_channels为CNN输出通道数
    def __init__(self, char_num, char_dim, window_size, out_channels):
        super(CharCNNMaxpooling, self).__init__()
        # 字符表向量，共有char_num个向量，每个维度为char_dim
        self.char_embed = nn.Embedding(num_embeddings=char_num, embedding_dim=char_dim)
        # 1个输入通道，out_channels个输出通道，过滤器大小为window_size*char_dim
        self.cnn = nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=(window_size, char_dim))

    # 输入char_ids为batch组文本，每个文本长度为seq_len，每个词含word_len个字符编号（0~char_num-1），输入维度为batch*seq_len*word_len
    # 输出res为所有单词的字符向量表示，维度是batch*seq_len*out_channels
    def forward(self, char_ids):
        # 根据字符编号得到字符向量，结果维度为batch*seq_len*word_len*char_dim
        x = self.char_embed(char_ids)
        # 合并前两维并变成单通道，结果维度（batch*seq_len）*1*word_len*char_dim
        x_unsqueeze = x.view(-1, x.shape[2], x.shape[3]).unsqueeze(1)
        # CNN，结果维度为（batch*seq_len）*out_channels*new_seq_len*1
        x_cnn = self.cnn(x_unsqueeze)
        # 删除最后一维，结果维度为（batch*seq）*out_channels*new_seq_len
        x_cnn_result = x_cnn.squeeze(3)
        # 最大池化，遍历最后一维求最大值，结果维度为（batch*seq_len）*out_channels
        res, _ = x_cnn_result.max(2)  # 
        # print('res_如下:')
        # print(res)
        # print('res_: {shape}'.format(shape=res.shape))
        return res.view(x.shape[0], x.shape[1], -1)  # x.shape[0] = batch, x.shape[1] = seq_len，不确定第2个维度是几，但是一定指定前两个维度的大小，所以这里用-1代替

In [41]:
# test

In [42]:
batch = 10  # batch
seq_len = 20  # 每个文本长度
word_len = 12  # 单词中字符个数
char_num = 26  # 字符表大小
char_dim = 10  # 字符向量长度
window_size = 3
out_channels = 8
char_cnn = CharCNNMaxpooling(char_num=char_num, char_dim=char_dim, window_size=window_size, out_channels=out_channels)
char_ids = torch.LongTensor(batch, seq_len, word_len).random_(0, char_num - 1)
print('char_ids: {shape}'.format(shape=char_ids.shape))
res = char_cnn(char_ids)
print('res: {shape}'.format(shape=res.shape))
print(res)

char_ids: torch.Size([10, 20, 12])
res: torch.Size([10, 20, 8])
tensor([[[ 0.6529,  0.8465,  0.4470,  ...,  1.4186,  0.8840,  0.5334],
         [ 0.3931,  1.1519,  0.6541,  ...,  0.8053,  1.0912,  0.9075],
         [ 1.1718,  1.2721,  0.8915,  ...,  1.2315,  0.8225,  0.5993],
         ...,
         [ 0.3707,  0.8362,  0.3265,  ...,  1.7615,  1.3467,  1.0405],
         [ 0.9671,  1.0019,  0.5029,  ...,  1.5247,  1.1616,  1.0269],
         [ 0.3161,  1.0389,  0.9601,  ...,  1.4771,  1.2479,  1.4900]],

        [[ 0.4622,  1.3565,  0.9146,  ...,  1.8216,  1.0728,  0.7849],
         [ 1.4459,  0.7593,  0.7322,  ...,  0.7574,  1.2987,  1.0229],
         [ 0.8910,  0.9636,  0.5123,  ...,  0.7922,  0.6371,  0.8367],
         ...,
         [ 1.5457,  1.1434,  0.7236,  ...,  1.3477,  0.6684,  1.3263],
         [ 0.3809,  1.1418,  0.3265,  ...,  0.7690,  1.0942,  0.7274],
         [ 0.8428,  0.8657,  0.9694,  ...,  1.2150,  1.0022,  1.0709]],

        [[ 0.7381,  0.8110,  0.8606,  ...,  1.7145, 

In [12]:
char_cnn

CharCNNMaxpooling(
  (char_embed): Embedding(26, 10)
  (cnn): Conv2d(1, 8, kernel_size=(3, 10), stride=(1, 1))
)

## 上下文编码

&emsp;&emsp;&emsp;&emsp;理解一个单词需要考虑它的上下文，很多具有多义性的单词需要考虑它周围的语句才能确定其明确含义。然而，基于词表的单词向量是固定的，不会随上下文变化。这就会导致同一个词在不同的语句中语义不同但其向量表示完全相同的情况。因此，编码层需要对每个单词生成上下文编码（contextual embedding）。这种编码会随着单词的上下文不同而发生改变，从而反映单词在当前语句中的含义。

&emsp;&emsp;&emsp;&emsp;在深度学习中，为了实现上下文语义的理解通常采用单词之间的信息传递。RNN是最常用的上下文编码生成结构，因为RNN利用相邻单词间的状态向量转移实现语义信息的传递。为了更有效地利用每个单词左右两个方向的语句信息，常采用双向循环神经网络（bidirectional RNN）获得上下文编码。而许多模型还采用了多层RNN，用于提取更高级的上下文语义，获得更好的效果。以下是多层双向RNN获得文本单词上下文编码的代码。

In [43]:
import torch
import torch.nn as nn

In [44]:
class ContextualEmbedding(nn.Module):
    # word_dim为词向量维度，state_dim为RNN状态维度，rnn_layer为RNN层数
    def __init__(self, word_dim, state_dim, rnn_layer):
        super(ContextualEmbedding, self).__init__()
        # 多层双向GRU
        self.rnn = nn.GRU(input_size=word_dim, hidden_size=state_dim, num_layers=rnn_layer, bidirectional=True, batch_first=True)
    
    # 输入x为batch组文本，每个文本长度为seq_len，每个词用一个word_dim维向量表示，输入维度为batch*seq_len*word_dim
    # 输出res为所有单词的上下文向量表示，维度是batch*seq_len*out_dim
    def forward(self, x):
        res, _ = self.rnn(x)  # 输出维度是batch*seq_len*out_dim，其中out_dim=2*state_dim，包含两个方向
        return res

In [45]:
# test

In [46]:
batch = 10
seq_len = 20
word_dim = 50
state_dim = 100
rnn_layer = 2
x = torch.randn(batch, seq_len, word_dim)
context_embed = ContextualEmbedding(word_dim=word_dim, state_dim=state_dim, rnn_layer=rnn_layer)
res = context_embed(x)
print(res.shape)
print(res)

torch.Size([10, 20, 200])
tensor([[[ 0.0080,  0.0752,  0.0691,  ..., -0.1804, -0.1286, -0.0760],
         [ 0.0345,  0.2315, -0.0525,  ..., -0.0794, -0.0285, -0.0648],
         [-0.0214,  0.2942,  0.0620,  ...,  0.0956,  0.0647, -0.1344],
         ...,
         [ 0.0417,  0.2764, -0.0351,  ...,  0.0393, -0.0262, -0.0384],
         [-0.0468,  0.1864, -0.0866,  ...,  0.1061, -0.0113, -0.0976],
         [ 0.0095,  0.1128, -0.2422,  ...,  0.0659, -0.0028, -0.0171]],

        [[ 0.0631,  0.0126, -0.0295,  ..., -0.0325,  0.1695, -0.0288],
         [ 0.0397, -0.0197, -0.1112,  ...,  0.0062,  0.0411, -0.0849],
         [-0.0527, -0.0169, -0.1902,  ..., -0.0864, -0.1091, -0.0711],
         ...,
         [ 0.0062,  0.1072,  0.0422,  ...,  0.0799, -0.4598, -0.2337],
         [-0.0610,  0.2202,  0.0625,  ...,  0.1482, -0.3998, -0.0943],
         [-0.1604,  0.2246,  0.0001,  ...,  0.0522, -0.2840, -0.0291]],

        [[-0.0154,  0.1051,  0.2525,  ..., -0.1313,  0.0892, -0.1459],
         [-0.0266, 

&emsp;&emsp;&emsp;&emsp;研究者进一步发现，在大规模自然语言处理任务上进行预训练，然后将预训练模型中的循环神经网络参数用用机器阅读理解，可以获得明显的性能提升，如CoVe模型在大量机器翻译数据上训练序列到序列模型，然后将编码器部分的循环神经网络用于SQuAD数据集，F1分数提高了近4%。

&emsp;&emsp;&emsp;&emsp;综上所述，在编码层中，每个问题单词由词表向量、命名实体向量、词性向量、字符编码、上下文编码组成，而每个文章单词除了以上5中向量外，还有精确匹配编码。

# 交互层

&emsp;&emsp;&emsp;&emsp;机器阅读理解模型在编码层中获得了文章和问题中单词的语义向量表示，但两部分的编码是基本独立的。为了获得最终答案，模型需要交互处理文章和问题中的信息。因此，模型在交互层中将文章和问题的语义信息融合，基于